<a href="https://colab.research.google.com/github/wangyiyang/RAG-Cookbook-Code/blob/main/ch04/prompt_templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy transformers torch sentence-transformers

In [1]:
"""
提示模板系统
实现多层次提示模板设计和动态优化
"""

import re
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from collections import defaultdict


@dataclass
class PerformanceMetrics:
    """性能指标"""
    accuracy: float = 0.0
    completeness: float = 0.0
    relevance: float = 0.0
    safety: float = 0.0
    response_time: float = 0.0


class PerformanceTracker:
    """性能跟踪器"""

    def __init__(self):
        self.prompt_history = defaultdict(list)
        self.metrics_cache = {}

    def record_performance(
        self,
        prompt_id: str,
        metrics: PerformanceMetrics
    ) -> None:
        """记录性能数据"""
        self.prompt_history[prompt_id].append(metrics)

        # 更新缓存的平均性能
        self.metrics_cache[prompt_id] = self._calculate_average_metrics(
            self.prompt_history[prompt_id]
        )

    def get_performance(self, prompt_id: str) -> Dict[str, float]:
        """获取性能数据"""
        if prompt_id in self.metrics_cache:
            return self.metrics_cache[prompt_id]

        # 返回默认值
        return {
            'accuracy': 0.7,
            'completeness': 0.7,
            'relevance': 0.8,
            'safety': 0.9,
            'response_time': 1.0
        }

    def _calculate_average_metrics(
        self,
        metrics_list: List[PerformanceMetrics]
    ) -> Dict[str, float]:
        """计算平均性能指标"""
        if not metrics_list:
            return {}

        total_metrics = {
            'accuracy': 0.0,
            'completeness': 0.0,
            'relevance': 0.0,
            'safety': 0.0,
            'response_time': 0.0
        }

        for metrics in metrics_list:
            total_metrics['accuracy'] += metrics.accuracy
            total_metrics['completeness'] += metrics.completeness
            total_metrics['relevance'] += metrics.relevance
            total_metrics['safety'] += metrics.safety
            total_metrics['response_time'] += metrics.response_time

        count = len(metrics_list)
        return {key: value / count for key, value in total_metrics.items()}


class AdvancedPromptTemplate:
    """高级提示模板系统"""

    def __init__(self):
        self.templates = {
            'standard': self.build_standard_template(),
            'analytical': self.build_analytical_template(),
            'creative': self.build_creative_template(),
            'factual': self.build_factual_template(),
            'technical': self.build_technical_template(),
            'comparison': self.build_comparison_template()
        }

        self.query_classifiers = {
            'analytical': ['分析', '原因', '影响', '评估', '比较', '研究'],
            'creative': ['创意', '设计', '建议', '方案', '策略', '创新'],
            'factual': ['什么是', '定义', '解释', '介绍', '概念', '含义'],
            'technical': ['实现', '算法', '代码', '技术', '方法', '流程'],
            'comparison': ['对比', '比较', '差异', '优缺点', '异同', 'vs']
        }

    def build_standard_template(self) -> str:
        """构建标准提示模板"""
        return """作为一个专业的AI助手，请基于以下可信信息准确回答用户问题。

## 📚 参考信息
{context}

## ❓ 用户问题
{question}

## 📋 回答要求
1. **准确性**：严格基于提供的参考信息进行回答
2. **完整性**：全面回答问题的各个方面
3. **可信度**：如果信息不足或存在不确定性，请明确说明
4. **来源引用**：明确标注信息来源
5. **通俗易懂**：使用清晰、专业但易理解的语言

## 🎯 回答格式
**答案**：[基于参考信息的详细回答]

**信息来源**：
- 来源1：[具体引用内容]
- 来源2：[具体引用内容]

**可信度评估**：[高/中/低] - [说明原因]

## 💬 您的回答："""

    def build_analytical_template(self) -> str:
        """构建分析型提示模板"""
        return """作为专业分析师，请基于提供的信息进行深度分析。

## 📊 分析材料
{context}

## 🔍 分析问题
{question}

## 🎯 分析框架
请按照以下框架进行系统性分析：

### 1. 🔎 现状分析
- 基于材料客观描述当前情况
- 识别关键事实和数据

### 2. 🧠 原因分析
- 深入挖掘问题的根本原因
- 分析内在逻辑和关联关系

### 3. 📈 影响评估
- 分析可能的短期和长期影响
- 评估对不同stakeholder的影响

### 4. 💡 建议方案
- 提出可行的解决方案
- 评估方案的可操作性

## ⚠️ 分析要求
- 逻辑清晰，层次分明
- 有理有据，避免主观臆断
- 如遇信息不足，明确指出局限性

## 📋 分析报告："""

    def build_creative_template(self) -> str:
        """构建创意型提示模板"""
        return """作为创新思维专家，请基于提供信息进行创意性思考。

## 💡 创意素材
{context}

## 🎨 创意需求
{question}

## 🚀 创意指导原则
1. **创新性**：提出新颖独特的想法
2. **实用性**：确保建议具有可操作性
3. **多样性**：从多个角度提供不同方案
4. **可行性**：考虑实施的现实条件

## 🎯 创意输出格式
### 💭 核心创意
[简明扼要地描述主要创意想法]

### 🔧 实施方案
- **方案A**：[详细描述]
- **方案B**：[详细描述]
- **方案C**：[详细描述]

### 📊 可行性分析
- **优势**：[列出主要优点]
- **挑战**：[指出潜在困难]
- **建议**：[提出具体建议]

## 🌟 您的创意方案："""

    def build_factual_template(self) -> str:
        """构建事实型提示模板"""
        return """作为知识专家，请基于权威信息准确回答用户的事实性问题。

## 📖 权威资料
{context}

## ❓ 事实询问
{question}

## 📝 回答标准
1. **精确性**：确保事实信息的准确无误
2. **权威性**：优先引用权威来源
3. **时效性**：注意信息的时间相关性
4. **完整性**：涵盖问题的关键要素

## 📋 标准回答格式
### 📌 核心事实
[简洁明确的核心答案]

### 📚 详细说明
[展开解释，提供更多背景信息]

### 🔗 相关信息
[补充相关的有用信息]

### 📄 信息来源
- [来源1]：[具体引用]
- [来源2]：[具体引用]

### ⚠️ 注意事项
[如有需要，说明信息的局限性或时效性]

## 💬 准确回答："""

    def build_technical_template(self) -> str:
        """构建技术型提示模板"""
        return """作为技术专家，请基于技术文档回答用户的技术问题。

## 🔧 技术文档
{context}

## 💻 技术问题
{question}

## 🎯 技术回答要求
1. **专业性**：使用准确的技术术语
2. **实用性**：提供可操作的技术指导
3. **清晰性**：适当解释复杂概念
4. **完整性**：涵盖实现的关键步骤

## 📋 技术回答格式
### 🎯 核心解答
[直接回答技术问题的核心]

### 🔍 技术细节
[详细的技术实现说明]

### 💡 最佳实践
[相关的最佳实践建议]

### ⚠️ 注意事项
[重要的注意事项和潜在风险]

### 🔗 相关技术
[相关技术或扩展阅读]

## 🛠️ 技术回答："""

    def build_comparison_template(self) -> str:
        """构建对比型提示模板"""
        return """作为对比分析专家，请基于提供信息进行客观比较分析。

## 📊 对比材料
{context}

## ⚖️ 对比问题
{question}

## 🎯 对比分析框架
### 📋 对比维度
请从以下维度进行系统对比：
- **功能特性**
- **性能表现**
- **使用场景**
- **优缺点**
- **成本效益**

### 📊 对比矩阵
| 维度 | 对象A | 对象B | 对象C |
|------|-------|-------|-------|
| [维度1] | [评价] | [评价] | [评价] |
| [维度2] | [评价] | [评价] | [评价] |

### 🏆 综合评估
- **最佳场景**：[不同对象的最适用场景]
- **选择建议**：[基于不同需求的选择建议]

## ⚖️ 客观对比分析："""

    def select_template(
        self,
        query: str,
        query_type: Optional[str] = None
    ) -> str:
        """根据查询类型选择合适的提示模板"""
        if query_type is None:
            query_type = self.classify_query_type(query)

        template = self.templates.get(query_type, self.templates['standard'])
        return template

    def classify_query_type(self, query: str) -> str:
        """查询类型智能分类"""
        query_lower = query.lower()

        # 计算每种类型的匹配分数
        type_scores = {}

        for query_type, keywords in self.query_classifiers.items():
            score = 0
            for keyword in keywords:
                if keyword in query_lower:
                    score += 1

            # 规范化分数
            type_scores[query_type] = score / len(keywords)

        # 选择最高分的类型
        best_type = max(type_scores.items(), key=lambda x: x[1])

        # 如果最高分太低，返回标准类型
        if best_type[1] < 0.1:
            return 'standard'

        return best_type[0]


class DynamicPromptOptimizer:
    """动态提示优化器"""

    def __init__(self):
        self.performance_tracker = PerformanceTracker()
        self.prompt_variants = {}
        self.technical_terms = self._load_technical_terms()

    def optimize_prompt(
        self,
        base_prompt: str,
        query: str,
        context: str
    ) -> str:
        """基于历史表现动态优化提示"""
        # 1. 获取历史表现数据
        prompt_id = self._get_prompt_id(base_prompt)
        historical_performance = self.performance_tracker.get_performance(prompt_id)

        # 2. 根据查询特征调整提示
        optimized_prompt = self.adjust_prompt_by_query(
            base_prompt, query, context
        )

        # 3. 添加个性化指导
        optimized_prompt = self._add_performance_guidance(
            optimized_prompt, historical_performance
        )

        return optimized_prompt

    def adjust_prompt_by_query(
        self,
        prompt: str,
        query: str,
        context: str
    ) -> str:
        """根据查询和上下文特征调整提示"""
        adjustments = []

        # 复杂查询需要更详细的指导
        if len(query.split()) > 10:
            adjustments.append("**注意**：这是一个复杂问题，请分步骤详细回答。")

        # 专业术语较多的查询
        if self.has_technical_terms(query):
            adjustments.append("**要求**：请解释专业术语，确保答案易于理解。")

        # 上下文信息较少
        if len(context.split()) < 100:
            adjustments.append("**提醒**：参考信息有限，如有不确定请明确说明。")

        # 包含时间敏感信息
        if self._contains_time_sensitive_info(query):
            adjustments.append("**重要**：请注意信息的时效性，标注数据的时间范围。")

        # 涉及数字或统计
        if re.search(r'\d+', query):
            adjustments.append("**精确性**：涉及数字信息，请确保数据准确性。")

        # 添加调整到提示末尾
        if adjustments:
            prompt += "\n\n## 🎯 特别指导\n" + "\n".join(adjustments)

        return prompt

    def _add_performance_guidance(
        self,
        prompt: str,
        performance: Dict[str, float]
    ) -> str:
        """添加基于历史性能的指导"""
        guidance = []

        if performance['accuracy'] < 0.8:
            guidance.append("**准确性提醒**：请特别关注信息的准确性和来源的可靠性。")

        if performance['completeness'] < 0.7:
            guidance.append("**完整性提醒**：请确保回答的完整性，涵盖问题的各个方面。")

        if performance['relevance'] < 0.8:
            guidance.append("**相关性提醒**：请紧扣问题主题，避免偏离重点。")

        if performance['safety'] < 0.9:
            guidance.append("**安全性提醒**：请确保回答内容安全，避免有害信息。")

        if guidance:
            prompt += "\n\n## ⚠️ 质量提醒\n" + "\n".join(guidance)

        return prompt

    def has_technical_terms(self, text: str) -> bool:
        """检测文本是否包含技术术语"""
        text_lower = text.lower()
        technical_count = sum(1 for term in self.technical_terms if term in text_lower)
        return technical_count > 2

    def _contains_time_sensitive_info(self, query: str) -> bool:
        """检测是否包含时间敏感信息"""
        time_keywords = ['最新', '现在', '目前', '当前', '今年', '最近', '实时']
        return any(keyword in query for keyword in time_keywords)

    def _get_prompt_id(self, prompt: str) -> str:
        """生成提示ID"""
        import hashlib
        return hashlib.md5(prompt.encode()).hexdigest()[:8]

    def _load_technical_terms(self) -> List[str]:
        """加载技术术语列表"""
        return [
            'algorithm', 'api', 'database', 'framework', 'library',
            'machine learning', 'neural network', 'deep learning',
            'artificial intelligence', 'rag', 'llm', 'transformer',
            '算法', '数据库', '框架', '机器学习', '神经网络',
            '深度学习', '人工智能', '检索增强', '大模型'
        ]


# 使用示例
if __name__ == "__main__":
    # 初始化提示模板系统
    template_system = AdvancedPromptTemplate()
    optimizer = DynamicPromptOptimizer()

    # 测试查询
    test_queries = [
        "什么是RAG技术？",
        "分析RAG技术的优缺点",
        "设计一个RAG系统的方案",
        "对比不同的检索算法",
        "RAG系统的技术实现方法"
    ]

    context = "RAG是检索增强生成技术，结合了信息检索和文本生成..."

    for query in test_queries:
        print(f"\n查询: {query}")
        print("=" * 50)

        # 分类查询类型
        query_type = template_system.classify_query_type(query)
        print(f"查询类型: {query_type}")

        # 选择模板
        template = template_system.select_template(query)

        # 优化提示
        optimized_prompt = optimizer.optimize_prompt(template, query, context)

        # 填充模板（示例）
        filled_prompt = optimized_prompt.format(
            context=context,
            question=query
        )

        print(f"\n优化后的提示（前200字符）:")
        print(filled_prompt[:200] + "...")
        print("\n" + "=" * 50)


查询: 什么是RAG技术？
查询类型: factual

优化后的提示（前200字符）:
作为知识专家，请基于权威信息准确回答用户的事实性问题。

## 📖 权威资料
RAG是检索增强生成技术，结合了信息检索和文本生成...

## ❓ 事实询问
什么是RAG技术？

## 📝 回答标准
1. **精确性**：确保事实信息的准确无误
2. **权威性**：优先引用权威来源
3. **时效性**：注意信息的时间相关性
4. **完整性**：涵盖问题的关键要素

## 📋 标准回答格式
#...


查询: 分析RAG技术的优缺点
查询类型: analytical

优化后的提示（前200字符）:
作为专业分析师，请基于提供的信息进行深度分析。

## 📊 分析材料
RAG是检索增强生成技术，结合了信息检索和文本生成...

## 🔍 分析问题
分析RAG技术的优缺点

## 🎯 分析框架
请按照以下框架进行系统性分析：

### 1. 🔎 现状分析
- 基于材料客观描述当前情况
- 识别关键事实和数据

### 2. 🧠 原因分析  
- 深入挖掘问题的根本原因
- 分析内在逻辑和关联关系...


查询: 设计一个RAG系统的方案
查询类型: creative

优化后的提示（前200字符）:
作为创新思维专家，请基于提供信息进行创意性思考。

## 💡 创意素材
RAG是检索增强生成技术，结合了信息检索和文本生成...

## 🎨 创意需求
设计一个RAG系统的方案

## 🚀 创意指导原则
1. **创新性**：提出新颖独特的想法
2. **实用性**：确保建议具有可操作性
3. **多样性**：从多个角度提供不同方案
4. **可行性**：考虑实施的现实条件

## 🎯 创意输出格...


查询: 对比不同的检索算法
查询类型: technical

优化后的提示（前200字符）:
作为技术专家，请基于技术文档回答用户的技术问题。

## 🔧 技术文档
RAG是检索增强生成技术，结合了信息检索和文本生成...

## 💻 技术问题
对比不同的检索算法

## 🎯 技术回答要求
1. **专业性**：使用准确的技术术语
2. **实用性**：提供可操作的技术指导
3. **清晰性**：适当解释复杂概念
4. **完整性**：涵盖实现的关键步骤

## 📋 技